In [1]:
import config
import os
import cv2
import albumentations as A
from tqdm import tqdm


In [2]:
# Caminhos
input_image_dir = config.dataset_path+'train/image'
input_mask_dir  = config.dataset_path+'train/label'
output_image_dir = config.dataset_path+'train_aug/image'
output_mask_dir  = config.dataset_path+'train_aug/label'

# Criar diretórios se não existirem
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Número de aumentações por imagem
N = 20

# Transformações
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    #A.Rotate(limit=45, p=0.5),
    A.ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=0.1,
            rotate_limit=30,
            p=0.7,
            border_mode=cv2.BORDER_REFLECT
    ),
    A.RandomBrightnessContrast(p=0.5),
    A.ElasticTransform(p=0.2),
    A.GaussianBlur(p=0.3),
    A.GridDistortion(p=0.2),
    A.GaussNoise(p=0.2),
])

# Listar imagens
image_filenames = sorted(os.listdir(input_image_dir))

# Aplicar aumentações
for img_name in tqdm(image_filenames):
    img_path = os.path.join(input_image_dir, img_name)
    mask_path = os.path.join(input_mask_dir, img_name)

    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Salvar imagem original
    cv2.imwrite(os.path.join(output_image_dir, f"{img_name[:-4]}_orig.png"), image)
    cv2.imwrite(os.path.join(output_mask_dir, f"{img_name[:-4]}_orig.png"), mask)

    for i in range(N):
        augmented = transform(image=image, mask=mask)
        aug_img = augmented['image']
        aug_mask = augmented['mask']

        out_img_path = os.path.join(output_image_dir, f"{img_name[:-4]}_aug{i}.png")
        out_mask_path = os.path.join(output_mask_dir, f"{img_name[:-4]}_aug{i}.png")

        cv2.imwrite(out_img_path, aug_img)
        cv2.imwrite(out_mask_path, aug_mask)


/home/alan/miniconda3/envs/pytorch/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
100%|██████████| 30/30 [00:18<00:00,  1.66it/s]
